# REACTIVE COVERING

<p>G23<br>
FERNÁNDEZ DE LA INFIESTA, Álvaro<br>
LAHOZ GAITX, Pau<br>
MUÑOZ MORALES, Víctor<br>
08/12/2022</p>

## Introduction

<p>This <em>.ipynb</em> document contains the codes of the different stages of the project, introduced by a brief description of the approach considered in each case. The following sections are presented:<br></p>

<p>1. Introduction: Import of the libraries and definition of the <em>showSolution</em> function.<br>
2. Stage 1: Familiarisation with the input data and first approach to cost and quality calculation.<br>
3. Stage 2: Solve the problem with full pass selection and test a MILP approach with DOCPLEX to optimise the cost.<br>
4. Stage 3: Solve the problem but now introducing variable booking time.<br>
5. Stage 4: Solve the multivariate problem, optimisation of cost and quality based on an adjustable 	$\alpha$ parameter.<br>
6. Stage 5: Implement the problem by applying a handmade greedy algorithm, no DOCPLEX.<br>
7. Display the saved results for each desired stage, input data file and $\alpha$ value (stage 4 only).</p>

In [ ]:
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cf
import math
from docplex.mp.model import Model

import random
import json

def showSolution(data,solution):
    
    # print the booking strategy
    print("Bookings:")
    for booking in solution['Bookings']:
        print(booking)

    # show the passes
    minVisiDuration = data['minVisiDuration']
    AOIs = data['AOIs']
    nAOIs = len(AOIs)
    accesses = data['Accesses']
    satPasses = data['Passes']
    nPasses = len(satPasses)
    
    ncols = 3
    nrows = math.ceil(nPasses / ncols)
    proj = ccrs.PlateCarree()
    fig, axs = plt.subplots(nrows=nrows,ncols=ncols, subplot_kw={'projection': proj},figsize=(30,80))
    axs = axs.flatten()

    for i in range(nPasses):
        satPass = satPasses[i]
        ax = axs[i]
        ax.set_extent([-15, 25, 35, 60])
        ax.stock_img()
        ax.add_feature(cf.COASTLINE, lw=1)
        ax.add_feature(cf.BORDERS)
        #plt.gcf().set_size_inches(15, 15)    
        for aoi in  AOIs:
            ax.plot([aoi['lon']], [aoi['lat']], 'kx')
        ax.set_title("Pass #{} [{},{}]".format(i,satPass['startDate'],satPass['endDate']))
    
        booking = None
        for bk in solution['Bookings']:
            if bk['passId'] == i:
                booking = bk            
                break
        
        for accessId in satPass['accessIds']:
            access = accesses[accessId]
            aoi = AOIs[access['aoiId']]            
            ax.plot([aoi['lon']], [aoi['lat']], 'rx')
            # show the reservations
            if (booking != None) and (booking['bookingStart'] <= access['end'] - minVisiDuration) and (access['start'] + minVisiDuration <= booking['bookingEnd']):
                ax.plot([aoi['lon']], [aoi['lat']], 'bo')

    plt.show()

## Stage 1

<p>This first section has served to become familiar with the input data of the project, defined in the 3 <em>.json</em> files for the three different reactivities of the problem: 2, 4 and 12. The <em>very basic solver</em> that came in the default code has been maintained and the strategy to calculate the cost and quality/satisfaction score of the objectives has been established. These two strategies are explained below:</p>

- <strong>Cost</strong>: (Total number of passes carried out x Unit cost per pass reservation) + (Time sum of all passes made) x (Cost per pass time unit)
- <strong>Quality</strong>: Total number of observations requests satisfied / Total number of observations requests. A satisfied observation request is detected if it is found in the accesses of the satellite, but also if the satellite is active (booked) during the duration of the observation request. Each satisfied observation adds 1 to the total number of observations requests satisfied.

<p>The obtained results are saved in the file <em>mySolutionStage1{fileName}.json</em></p>

In [ ]:
# Import the data
fileName = "data2"
dataFile = open('data/{0}.json'.format(fileName))  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nObsRequestsToCover = nObsRequests
nPasses = len(satPasses)
nGoals = len(goals)

# Very basic solver
# We start considering all the AOIs uncovered
covered = [False for i in range(nObsRequests)]
remainingSatPassesIds = list(range(nPasses))
random.shuffle(remainingSatPassesIds)
selectedPassIds = []
for i in remainingSatPassesIds:
    # Try to select the ith path
    satPass = satPasses[i]
    keepPass = False
    for accessId in satPass['accessIds']:
        access = accesses[accessId]
        for j in range(nObsRequests):
            obsRequest = obsRequests[j]
            if not covered[j] and accessId in obsRequest['accessIds']:
                covered[j] = True
                keepPass = True
                nObsRequestsToCover -= 1
                
    if keepPass:
        selectedPassIds.append(i)
            
    if nObsRequestsToCover == 0:
        break

# Computation of the total cost
timeUses = [satPasses[id]['end'] - satPasses[id]['start'] for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering: It is defined as the relation of the 
# requests covered over the total amount of requests
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for passId in selectedPassIds:
                if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                    if (requestStart<=satPasses[passId]['end']-30) and (requestEnd>=satPasses[passId]['start']+30):
                        quality +=1
                        break
        aux += goals[goalId]['nSteps']
        
quality /= nObsRequests

# Export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": satPass['start'], "bookingEnd": satPass['end']})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolutionStage1{0}.json".format(fileName), "w")
jsonFile.write(jsonString)
jsonFile.close()

# Presentation of the results
print("------------------------------RESULTS------------------------------")
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))
print("Bookings:")
for booking in bookings:
    print(booking)

## Stage 2

<p>In this stage, the problem is posed by considering a selection of complete passes, testing a
MILP approach with DOCPLEX. To do this, the <em>reactiveCovering</em> model is created and the binary variable <em>selectedPass</em> is called, which has a box reserved for all the existing passes and, after optimisation, stores 1 for the booked passes and 0 for the rest. The objective function is the cost, the value to be optimised, which in this case is calculated with the established start and end times of the satellite passes booked during the optimisation.</p>

<p>Two constraints are set for this case: respecting a minimum access duration of 30s and ensuring that all the goals are covered by at least one access of a pass. For the latter, the <em>covering</em> function has been created and implemented inside the loop to find the satisfied accesses.</p>

<p>Once solved the MILP problem, the cost and quality are computed as mentioned above, but now considering the start and end times of the selected passes after the optimisation. The obtained results are saved in the file <em>mySolutionStage2{fileName}.json</em></p>

In [ ]:
# Import the data
fileName = "data12"
dataFile = open('data/{0}.json'.format(fileName))  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nPasses = len(satPasses)
nGoals = len(goals)

# OPTIMISATION MODEL
# Model creation
model = Model("reactiveCovering")

# Variables
selectedPass = model.binary_var_list(range(nPasses), name='selectedPass')

# Objective function: Minimisation of the cost
model.minimize(model.sum(selectedPass)*data['fixedPassCost']
    + model.sum((satPasses[id]['end'] - satPasses[id]['start'])*selectedPass[id] 
    for id in range(nPasses))*data['passCostPerTimeUnit'])

# Contraints:
# 1) Ensuring that all the goals are covered by at least one access of a pass
def covering(requestStart, requestEnd, passId, obsReqId):
    if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
        if requestStart<=(satPasses[passId]['end'] - 30) and requestEnd>=(satPasses[passId]['start'] + 30):
            return selectedPass[passId]
    return 0

# Checking for all the observation requests
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            coverings = [covering(requestStart, requestEnd, passId, obsReqId) for passId in range(nPasses)]
            sum_covering = model.sum(coverings)
            if not str(sum_covering) == "0":
                model.add_constraint(sum_covering >= 1)
        aux += goals[goalId]['nSteps']

# Solve:
# Computation time limit
model.set_time_limit(300)

model.print_information()

# Call of the solver
solution = model.solve(log_output=True)

# Getting the solution
selectedPassIds = []
if solution is None:
    raise Exception("No solution")
else:
    for i in range(nPasses):
        if selectedPass[i].solution_value == 1:
            selectedPassIds.append(i)

# Computation of the total cost
timeUses = [satPasses[id]['end'] - satPasses[id]['start'] for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering: It is defined as the relation of the observation
# requests covered over the total amount of observation requests
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for passId in selectedPassIds:
                if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                    if (requestStart<=satPasses[passId]['end']-30) and (requestEnd>=satPasses[passId]['start']+30):
                        quality +=1
                        break
        aux += goals[goalId]['nSteps']
quality /= nObsRequests

# Export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": satPass['start'], "bookingEnd": satPass['end']})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolutionStage2{0}.json".format(fileName), "w")
jsonFile.write(jsonString)
jsonFile.close()

# Presentation of the results
print("------------------------------RESULTS------------------------------")
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))
print("Bookings:")
for booking in bookings:
    print(booking)


## Stage 3

<p>This section considers the problem posed previously but, unlike the previous stage, now the satellite booking times are variable. This implies the introduction of two new continuous variables, <em>tStart</em> and <em>tEnd</em>, referring to the start and end times of each satellite booking.</p>

<p>These two variables play a role in the solver at different points, such as in the cost objective function, in the constraint of an access duration greater than 30s, or in the constraint ensuring that all the goals are covered by at least one access of a pass. For the latter, a function called <em>accessings</em> has been defined, which is implemented in the observations requests check, process already mentioned above for the calculation of the quality.</p>

<p>On the other hand, two temporary constraints have also been added. These guarantee that the satellite booking times can start before or end after the observation request delimited times, but always guaranteeing the constraint of 30 seconds within the observation request duration.</p>

<p>Once solved the MILP problem, the cost and quality are computed as mentioned above, using the <em>tStart</em> and <em>tEnd</em> values obtained after the optimisation. The obtained results are saved in the file <em>mySolutionStage3{fileName}.json</em></p>

In [ ]:
# Import the data
fileName = "data4"
dataFile = open('data/{0}.json'.format(fileName)) 
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nPasses = len(satPasses)
nGoals = len(goals)

# OPTIMISATION MODEL
# Model creation
model = Model("reactiveCovering")

# Definition of the boundary lists for the variables definition
lbs = []
ube = []
for i in range(nPasses):
    lbs.append(satPasses[i]['start'])
    ube.append(satPasses[i]['end'])

# Variables definition
selectedPass = model.binary_var_list(range(nPasses), name='selectedPass')
tStart = model.continuous_var_list(range(nPasses), lb=lbs, name='tStart')
tEnd = model.continuous_var_list(range(nPasses), ub=ube, name='tEnd')

# Objective function: Minimisation of the cost
model.minimize(model.sum(selectedPass)*data['fixedPassCost'] 
               + (model.sum((tEnd[id] - tStart[id])for id in range(nPasses)) 
                  - 30*(nPasses - model.sum(selectedPass)))*data['passCostPerTimeUnit'])  

# Contraints:
for passId in range(nPasses):
    model.add_constraint((tEnd[passId] - tStart[passId]) >= 30)
    
# 1) Ensuring that all the goals are covered by at least one access of a pass
def accessing(requestStart, requestEnd, passId, obsReqId):
    if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
        if requestStart<=(satPasses[passId]['end'] - 30) and requestEnd>=(satPasses[passId]['start'] + 30):
            return selectedPass[passId]
    return 0

# Checking for all the observation requests
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            accessings = [accessing(requestStart, requestEnd, passId, obsReqId) for passId in range(nPasses)]
            sum_accessing = model.sum(accessings)
            if not str(sum_accessing) == "0":
                # 1) Ensuring that all the goals are accessed
                model.add_constraint(sum_accessing >= 1)
                for passId in range(nPasses):
                    if not str(accessings[passId]) == "0":
                        # 2) tStart covering constraint
                        model.add_constraint(requestStart <= tEnd[passId]-30)
                        # 3) tEnd covering constraint
                        model.add_constraint(requestEnd >= tStart[passId]+30)
        aux += goals[goalId]['nSteps']


# Solve
# Time limitation
model.set_time_limit(60)
model.print_information()

# Call of the solver
solution = model.solve(log_output=True)

# solution.display()

# Getting the solution
selectedPassIds = []
if solution is None:
    raise Exception("No solution")
else:
    for i in range(nPasses):
        if selectedPass[i].solution_value == 1:
            selectedPassIds.append(i)

# Computation of the total cost
timeUses = [tEnd[id].solution_value - tStart[id].solution_value for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + model.sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering: It is defined as the relation of the observation
# requests covered over the total amount of observation requests
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for passId in selectedPassIds:
                if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                    if (requestStart<=tEnd[passId].solution_value-30) and (requestEnd>=tStart[passId].solution_value+30):
                        quality +=1
                        break
        aux += goals[goalId]['nSteps']
quality /= nObsRequests

# Export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": round(tStart[i].solution_value), "bookingEnd": round(tEnd[i].solution_value)})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolutionStage3{0}.json".format(fileName), "w")
jsonFile.write(jsonString)
jsonFile.close()

# Presentation of the results
print("------------------------------RESULTS------------------------------")
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))
print("Bookings:")
for booking in bookings:
    print(booking)


## Stage 4: Multi-criteria optimisation

<p>This last stage considers the multicriteria problem, optimising cost and quality based on an adjustable parameter $\alpha$ (from 0 to 1). In this case, the <em>tStart</em> and <em>tEnd</em> variables have not been taken into account, the only variable to be solved by the solver has been the binary list <em>selectedPass</em>.</p> 

<p>The objective function is the following: ($\alpha$*cost/maxCost)+(1-$\alpha$)(1-quality)</p>

<p>Once solved the MILP problem, the cost and quality are computed and the obtained results are saved in the file <em>mySolutionStage4{fileName}{alpha}.json</em></p>

In [ ]:
# Import the data
fileName = "data2"
dataFile = open('data/{0}.json'.format(fileName))  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nPasses = len(satPasses)
nGoals = len(goals)

# Definition of the adjustable alpha parameter
alpha = 1.0

# OPTIMISATION MODEL
# Model creation
model = Model("reactiveCovering")

# Definition of the boundary lists for the variable definition
lbs = []
ube = []
for i in range(nPasses):
    lbs.append(satPasses[i]['start'])
    ube.append(satPasses[i]['end'])

# Variables definition
selectedPass = model.binary_var_list(range(nPasses), name='selectedPass')

# Cost calculation
cost = (model.sum(selectedPass)*data['fixedPassCost']
    + model.sum((satPasses[id]['end'] - satPasses[id]['start'])*selectedPass[id] 
    for id in range(nPasses))*data['passCostPerTimeUnit'])

maxcost = 0
for id in range(nPasses):
    maxcost += fixedPassCost + (satPasses[id]['end'] - satPasses[id]['start'])*data['passCostPerTimeUnit'] 

# Quality calculation
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for i in range(len(selectedPass)):
                if any(accessIdObsReq in satPasses[i]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                    quality += selectedPass[i]
                    break
        aux += goals[goalId]['nSteps']  
quality /= nObsRequests

# Objective function:
model.minimize(alpha*cost/maxcost+(1-alpha)*(1-quality))

# Solve
# Time limitation
model.set_time_limit(60)

model.print_information()

# Call of the solver
solution = model.solve(log_output=True)

# solution.display()

# Getting the solution
selectedPassIds = []
if solution is None:
    raise Exception("No solution")
else:
    for i in range(nPasses):
        if selectedPass[i].solution_value == 1:
            selectedPassIds.append(i)

# Computation of the total cost
# timeUses = [tEnd[id].solution_value - tStart[id].solution_value for id in selectedPassIds]
timeUses = [satPasses[id]['end'] - satPasses[id]['start'] for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + model.sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering: It is defined as the relation of the observation 
# requests covered over the total amount of observation requests
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for passId in selectedPassIds:
                if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                     if (requestStart<=satPasses[passId]['end']-30) and (requestEnd>=satPasses[passId]['start']+30):
                        #if (requestStart<=tEnd[passId].solution_value-30) and (requestEnd>=tStart[passId].solution_value+30):
                        quality +=1
                        break
        aux += goals[goalId]['nSteps']
quality /= nObsRequests

# export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    # bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": tStart[i].solution_value, "bookingEnd": tEnd[i].solution_value})
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": satPass['start'], "bookingEnd": satPass['end']})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolutionStage4{0}Alpha{1}.json".format(fileName, int(alpha*100)), "w")
jsonFile.write(jsonString)
jsonFile.close()

# Presentation of the results
print("------------------------------RESULTS------------------------------")
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))
print("Bookings:")
for booking in bookings:
    print(booking)

## Stage 5: Greedy algorithm

<p>This handmade greedy algorithm returns a partially optimal solution (not the best), but with lower computational costs. The solver takes as input all the satellite passes and calculates the ratio: (Number of observation requests covered)/(cost). It then iterates and books the needed satellite passes in descending efficiency.</p>

<p>The results are saved in the file <em>mySolutionStage5{fileName}.json</em></p>

In [ ]:
# Import the data
fileName = "data2"
dataFile = open('data/{0}.json'.format(fileName))  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nPasses = len(satPasses)
nGoals = len(goals)

## Optimisation model:
# Objective function: ratio (obsReq covered)/(time)

# Lists from which we're gonna start popping items
satStart     = []
satEnd       = []
obsReqList   = [] # List of lists containing the obsReqs  for each sat

obsReqsToObs  = range(nObsRequests)
passesToCheck = []

for i in range(len(satPasses)):
    # Stock passes in new lists containing the data, start time and end time
    passesToCheck.append(satPasses[i])
    satStart.append(accesses[satPasses[i]['accessIds'][0]]['start'])
    satEnd.append(accesses[satPasses[i]['accessIds'][-1]]['end'])

    # Stock in a list of lists the obsReqs to whom the accessIds of the satellites refer to.
    # We optimise per obsReq, not accesReq
    obsReqList.append([])
    for aoiId in range(nAois):
        aux = 0
        for goalId in range(nGoals):
            for step in range(goals[goalId]['nSteps']):
                obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId # For all obsReqs
                for accessIdObs in satPasses[i]['accessIds']:
                    if accessIdObs in obsRequests[obsReqId]['accessIds'] and obsReqId not in obsReqList[i]:
                        obsReqList[i].append(obsReqId)

# Lists of variable to optimise
totObsReq = []
deltaT    = []
ratio     = []
for i in range(len(obsReqList)):
    totObsReq.append(len(obsReqList[i]))
    deltaT.append(satEnd[i]-satStart[i])
    ratio.append(totObsReq[i]/deltaT[i])

# We take first the pass with the higher ratio totObsReq/deltaT
bookedSats = []
startTimes = []
endTimes   = []

ite = 1

# Loop until there's no more obsReq to fulfill
while not sum(totObsReq) == 0:
    prevTot = totObsReq

    # Add the chosen satellite to the plan
    maxRatioIndex = ratio.index(max(ratio))
    bookedSats.append(satPasses[maxRatioIndex])

    startTimes.append(satStart[maxRatioIndex])
    endTimes.append(satEnd[maxRatioIndex])

    # Remove chosen satellite from the list of possible sats
    passesToCheck.pop(maxRatioIndex)
    satStart.pop(maxRatioIndex)
    satEnd.pop(maxRatioIndex)

    # Remove obsRequests from pool of requests to observe
    for obsId in obsReqList[maxRatioIndex]: # for each obsId within the list of obsIds of the chosen satellite
        for sat in passesToCheck: # for each satellite in the satellites to watch
            if obsId in obsReqList[passesToCheck.index(sat)]: # if obsId is inside the "minilist" of obsReqs 
                obsReqList[passesToCheck.index(sat)].pop(obsReqList[passesToCheck.index(sat)].index(obsId))

    obsReqList.pop(maxRatioIndex)
    
    totObsReq = []
    deltaT    = []
    ratio     = []
    for i in range(len(obsReqList)):
        totObsReq.append(len(obsReqList[i]))
        deltaT.append(satEnd[i]-satStart[i])
        ratio.append(totObsReq[i]/deltaT[i])

    if sum(totObsReq) == sum(prevTot) or not totObsReq:
        break

    ite += 1

duration = []
for i in range(len(startTimes)):
    duration.append(endTimes[i]-startTimes[i])
    
cost = data['fixedPassCost']*len(bookedSats) + sum(duration)*data['passCostPerTimeUnit']
print("Total cost: ", cost)

nonObserved = []
for i in obsReqList:
    for j in i:
        if not j in nonObserved:
            nonObserved.append(j)

print("quality: ", 1 - len(nonObserved)/nObsRequests)

selectedPassIds = []
for sat in bookedSats:
    selectedPassIds.append(satPasses.index(sat))

# export the solution to json
bookings = []
for i in selectedPassIds:
    satPass = satPasses[i]
    bookings.append({"passId": i, "passStart": satPass['start'], "passEnd": satPass['end'], "bookingStart": satPass['start'], "bookingEnd": satPass['end']})
jsonString = json.dumps({"Bookings": bookings})
jsonFile = open("mySolutionStage5{0}.json".format(fileName), "w")
jsonFile.write(jsonString)
jsonFile.close()

## Results presentation

<p>The next code displays the results obtained in the different stages of the project. The parameters <em>stage</em>, <em>fileName</em> and <em>alpha</em> must be modified depending on the desired result to be displayed.</p>

<p><strong>ATTENTION!</strong> You must previously have the <em>.json</em> files with the results in the main folder of the project. If these files are not there, you have to run the desired stage, with the desired filename and $\alpha$ value (only for stage 4).</p>

In [ ]:
stage = 3
fileName = "data4"
alpha = 0.5

# Import the data
dataFile = open('data/{0}.json'.format(fileName))  
data = json.load(dataFile)

# Load data
obsRequests = data['ObservationRequests']
fixedPassCost = data['fixedPassCost']
minVisiDuration = data['minVisiDuration']
aois = data['AOIs']
accesses = data['Accesses']
referenceDate = data['referenceDate']
passCostPerTimeUnit = data['passCostPerTimeUnit']
goals = data['Goals']
satPasses = data['Passes']

# Lengths of lists
nAois = len(aois)
nObsRequests = len(obsRequests)
nPasses = len(satPasses)
nGoals = len(goals)

# Import the solution
if stage == 4:
    solutionFile = open("mySolutionStage4{0}Alpha{1}.json".format(fileName, int(alpha*100)))  
else:
    solutionFile = open("mySolutionStage{0}{1}.json".format(stage,fileName))  
solution = json.load(solutionFile)

# Reading of the solution
bookings = solution["Bookings"]
selectedPassIds = []
tStart = [0] * nPasses
tEnd = [0] * nPasses
for i in range(len(bookings)):
    selectedPassIds.append(bookings[i]['passId'])
    tStart[bookings[i]['passId']] = bookings[i]['bookingStart']
    tEnd[bookings[i]['passId']] = bookings[i]['bookingEnd']

# Computation of the total cost
timeUses = [tEnd[id] - tStart[id] for id in selectedPassIds]
totalCost = len(selectedPassIds)*data['fixedPassCost'] + sum(timeUses)*data['passCostPerTimeUnit']

# Computation of the quality of the covering: It is defined as the relation of the 
# requests covered over the total amount of requests
quality = 0
for aoiId in range(nAois):
    aux = 0
    for goalId in range(nGoals):
        for step in range(goals[goalId]['nSteps']):
            obsReqId = aoiId + nAois*aux + step + (goals[goalId]['nSteps'] - 1)*aoiId
            requestStart = step*goals[goalId]['duStep']
            requestEnd = step*goals[goalId]['duStep'] + goals[goalId]['rctHorizon']
            for passId in selectedPassIds:
                if any(accessIdObsReq in satPasses[passId]['accessIds'] for accessIdObsReq in obsRequests[obsReqId]['accessIds']):
                    if (requestStart<=tEnd[passId]-30) and (requestEnd>=tStart[passId]+30):
                        quality +=1
                        break
        aux += goals[goalId]['nSteps']
        
quality /= nObsRequests

# Presentation of the results
print("IDs of the {0} selected passes: {1}".format(len(selectedPassIds), selectedPassIds))
print("Total cost of the covering: {0}".format(totalCost))
print("Quality of the covering: {0}".format(quality))

showSolution(data,solution)